In [92]:
# importing src directory
import sys
sys.path.append('..')
# experiment imports
import os
import math
import numpy as np
import random
from datetime import datetime as dt
from scipy.stats import truncnorm
from scipy import integrate
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import shapiro
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_pacf
from scipy.optimize import minimize
# project imports
from amm.amm import AMM, SimpleFeeAMM
from amm.fee import TriangleFee, PercentFee, NoFee
# data imports
from data.kaiko import fetch_data
from api_key.my_api_key import api_key

In [93]:

def gbm_assumption_test(log_returns):
    adf_result = adfuller(log_returns) # check for stationarity
    print("ADF Statistic:", adf_result[0]) # check for stationarity
    print("p-value:", adf_result[1])
    print("Critical Values:", adf_result[4])
    print("stationary:", adf_result[1] <= 0.05)
    # if adf_result[1] > 0.05:  # if not stationary, iteratively difference until achieved
    #     for d in range(1, max_lag + 1):
    #         diff_data = diff(log_returns, k_diff=d)
    #         adf_result = adfuller(diff_data)
    #         print(f"ADF result after differencing level {d}: {adf_result[0]}, p-value: {adf_result[1]}")
    #         if adf_result[1] <= 0.05:
    #             print("Achieved stationarity with differencing level:", d)
    #             diff_data = diff_data
    #             break
    shapiro_result = shapiro(log_returns) # check for normality
    print("Shapiro-Wilk Test Statistic:", shapiro_result[0])
    print("p-value:", shapiro_result[1])
    print("normal:", shapiro_result[1] > 0.05)
    lb_result = acorr_ljungbox(log_returns, lags=[10], return_df=True) # check for independence (autocorrelation)
    print("Ljung-Box test:")
    print(lb_result)
    print("independent:", lb_result['lb_pvalue'].iloc[0] > 0.05)
    if lb_result['lb_pvalue'].iloc[0] < 0.05: # if autocorrelation detected, adjust
        print("Autocorrelation detected:") 
        plot_pacf(log_returns, lags=40) # plot partial autocorrelation function
        plt.title('Partial Autocorrelation Function (PACF)')
        plt.show()

In [94]:

# Define the log-likelihood function
def neg_log_likelihood(params, log_returns):
    """
    calculate negative log likelihood of a normal distribution for calibrating GBM
    params: tuple, mu and sigma
    """
    gbm_assumption_test(log_returns) # test gbm assumptions
    mu, sigma = params # define mu and sigma
    estimated_mu = np.mean(log_returns) # estimate mu
    estimated_var = np.sum((log_returns - estimated_mu)**2) / len(log_returns) # estimate variance
    return 0.5 * len(log_returns) * np.log(2 * np.pi * estimated_var) + 0.5 / estimated_var * np.sum((log_returns - mu)**2) # return negative log likelihood

In [95]:
def calibrate_gbm(data, frequency, T, N, type, max_lag=10, alpha=0.05):
    """
    calibrate geometric brownian motion
    
    calibrate gbm model by pulling data from kaiko api

    data (pd.DataFrame): price data
    freq (str): frequency of data (1h, 1d, 1w)
    T (float): terminal time
    N (int): number of time steps
    type (str): type of calibration (reg, mle)
    max_lag (int): maximum lag for autocorrelation test (default=10)
    alpha (float): significance level for hypothesis tests (default=0.05)

    return numpy.ndarray: simulated gbm path
    """
    if type == "reg":
        returns = np.log(data / data.shift(1)) # get returns
        gbm_assumption_test(returns) # test gbm assumptions
        mu = returns.mean() * 252  # annualized return
        sigma = returns.std() * 252 ** 0.5 # annualized volatility
        print(f'Estimated {frequency} Mu:', mu, 'Estimated Annualized Mu:', mu * 365.25)
        print(f'Estimated {frequency} Sigma:', sigma, 'Estimated Annualized Sigma:', sigma * 365.25**0.5)
        S0 = data.iloc[-1] # get LAST price in series
        dt = T / N # time step size
        t = np.linspace(0, T, N)
        W = np.random.standard_normal(size=N)
        W = np.cumsum(W) * np.sqrt(dt)  # Standard Brownian motion
        X = (mu - 0.5 * sigma**2) * t + sigma * W 
        S = S0 * np.exp(X)  # Geometric Brownian motion
        
        return S
    elif type == "mle":
        log_returns = np.log(1 + data['price'].pct_change().dropna()) # calculate log returns
        result = minimize(neg_log_likelihood, [0.05, 0.2], bounds=((None, None), (1e-4, None))) # minimize the negative log-likelihood
        mu = result.x[0] * 365.25 # annualize mu
        sigma = result.x[1] * 365.25**0.5 # annualize sigma
        print(f'Estimated {frequency} Mu:', result.x[0], 'Estimated Annualized Mu:', mu) # using 365.25 instead of 252 bcs operate 24/7
        print(f'Estimated {frequency} Sigma:', result.x[1], 'Estimated Annualized Sigma:', sigma)
        S0 = data.iloc[-1] # get LAST price in series
        dt = T / N # time step size
        t = np.linspace(0, T, N)
        W = np.random.standard_normal(size=N)
        W = np.cumsum(W) * np.sqrt(dt)  # standard BM
        X = (mu - 0.5 * sigma**2) * t + sigma * W 
        S = S0 * np.exp(X)  # GBM
        
        return S

In [96]:
def get_gbm_data(asset, start_date, end_date, freq, api_key):
    """
    get gbm data from kaiko api or local storage

    asset (str): asset symbol
    start_date (str): start date of data
    end_date (str): end date of data
    freq (str): frequency of data (1h, 1d, 1w)
    api_key (str): kaiko api key

    return pd.DataFrame: price data
    """
    # check if data exists, if not fetch data
    if os.path.exists(f"/analyze/crypto_data/{asset}-usd_{start_date}_{end_date}_{freq}.csv"):
        data =  pd.read_csv(f"/analyze/crypto_data/{asset}-usd_{start_date}_{end_date}_{freq}.csv")["price"]
    else: data = fetch_data(api_key, asset+"-usd", start_date, end_date, freq)
    return data

In [97]:

def sim1(n, pair, start_dt, end_dt, frequency):
    """
    simulate AMM market with data calibrated GBM for external oracles and trading agents
    n (int): number of simulations
    pair (str): asset pair for data (e.g. btc-eth)
    asset1_n (int): number of asset1 tokens
    asset2_n (int): number of asset2 tokens
    start_dt (str): start date for data (YYYY-MM-DD)
    end_dt (str): end date for data (YYYY-MM-DD)
    frequency (str): frequency of data (1h, 1d, 1w)
    return list: list of dataframes for each simulation 
    """

    # # SIM STORAGE # #
    # create list to store dfs from each simulation of amms
    sim_amm_dfs= []
    sim_amms = []
    # parse asset1 and asset2, create USD denominated pairs
    asset1 = pair.split("-")[0] 
    asset2 = pair.split("-")[1]

    # # DATA & GBM CALIBRATION # #
    data = get_gbm_data(asset1, start_dt, end_dt, frequency, api_key) # get data for asset1
    difference = dt.strptime(end_dt, '%Y-%m-%dT%H:%M:%SZ') - dt.strptime(start_dt, '%Y-%m-%dT%H:%M:%SZ')
    T_years = difference.days / 365.25  # using 365.25 to account for leap years
    n_timesteps = len(data) # number of timesteps in data
    asset1_data = get_gbm_data(asset1, start_dt, end_dt, frequency, api_key) # get data for asset1
    asset2_data = get_gbm_data(asset2, start_dt, end_dt, frequency, api_key) # get data for asset2
    print(asset1_data)
    asset1_data['timestamp'] = pd.to_datetime(asset1_data['timestamp'], unit='ms') # convert timestamp to datetime
    asset2_data['timestamp'] = pd.to_datetime(asset2_data['timestamp'], unit='ms')
    asset1_data['price'] = pd.to_numeric(asset1_data['price']) # convert price to numeric
    asset2_data['price'] = pd.to_numeric(asset2_data['price'])
    marketDF = pd.merge(asset1_data, asset2_data, on='timestamp', how='inner', suffixes=("_" + asset1, "_" + asset2)) # merge dataframes on timestamp saving price for each asset denominated in USD for storing AMM market data
    # calculate market ratio of asset1/asset2
    marketDF[f'mrkt_{asset1}/{asset2}'] = marketDF[f'price_{asset1}'] / marketDF[f'price_{asset2}'] 
    # add columns for trade tracking (amm ratio, inventory, averages)
    new_cols = [f'amm_{asset1}/{asset2}', f'{asset1}_inv', f'{asset2}_inv', 'L_inv']
    marketDF = marketDF.assign(**{col: None for col in new_cols})

    # # MARKET MOVING AVERAGES # #
    # add columns for moving averages
    marketDF[f'20mavg_{asset1}'] = marketDF[f'price_{asset1}'].rolling(window=20).mean()
    marketDF[f'50mavg_{asset1}'] = marketDF[f'price_{asset1}'].rolling(window=50).mean()
    marketDF[f'200mavg_{asset1}'] = marketDF[f'price_{asset1}'].rolling(window=200).mean()
    marketDF[f'20mavg_{asset2}'] = marketDF[f'price_{asset2}'].rolling(window=20).mean()
    marketDF[f'50mavg_{asset2}'] = marketDF[f'price_{asset2}'].rolling(window=50).mean()
    marketDF[f'200mavg_{asset2}'] = marketDF[f'price_{asset2}'].rolling(window=200).mean()

    # # TIME SERIES SIMULATIONS # #
    for simulation in range(n): # for each simulation create new set of amms & run new set of trades
        market = marketDF.copy() # create new market df for each simulation
        nofeeAMM = SimpleFeeAMM(fee_structure = NoFee())  # setup amms to simulate
        percentAMM = SimpleFeeAMM(fee_structure = PercentFee(0.01))
        triAMM = SimpleFeeAMM(fee_structure = TriangleFee(0.003, 0.0001, -1)) 
        amm_cols = [f'{asset1}_inv', f'{asset2}_inv', 'L_inv', f'{asset1}', f'{asset2}', 'L', f'F{asset1}', f'F{asset2}', 'FL'] # setup new set of dfs to save simulations
        percentDF = pd.DataFrame(columns=amm_cols)
        nofeeDF = pd.DataFrame(columns=amm_cols)
        triDF = pd.DataFrame(columns=amm_cols)
        amms = [(nofeeAMM, nofeeDF), (percentAMM, percentDF), (triAMM, triDF)] # store pairs of amm type & df for updating
        asset1_gbm = calibrate_gbm(data, frequency, T_years, n_timesteps, "mle") # calibrate gbm for asset1 w/ MLE
        asset2_gbm = calibrate_gbm(data, frequency, T_years, n_timesteps, "mle") # calibrate gbm for asset2 w/ MLE
        asset1_data['gbm_price'] = asset1_gbm # add gbm price to df
        asset2_data['gbm_price'] = asset2_gbm # add gbm price to df
        
        # # SIMULATION # #
        for t in range(n_timesteps): # iterate over each timestep in crypto market data
            if marketDF[f'amm_{asset1}/{asset2}'][t] > (marketDF[f'market_{asset1}/{asset2}'][t] * 1.005): # rule-based arbitrage agents in the market
                asset_out, asset_in, asset_in_n = asset1, asset2, random.choice(list(range(1, 50))) # modeling market efficiency
            if (marketDF[f'amm_{asset1}/{asset2}'][t] * 1.005) < marketDF[f'market_{asset1}/{asset2}'][t]:
                asset_out, asset_in, asset_in_n = asset2, asset1, random.choice(list(range(1, 50)))
            else: continue
            for amm, df in amms: # update market data with amm data
                succ, info = amm.trade_swap(asset_out, asset_in, asset_in_n) # call trade for each AMM
                new_row = {f'{asset1}_inv': amm.portfolio[asset1], f'{asset2}_inv': amm.portfolio[asset2], 'LInv': amm.portfolio['L'], # add trade info to df
                        asset1: info['asset_delta'][asset1], f'{asset2}': info['asset_delta'][asset2], 'L': info['asset_delta']['L'], 
                        f'F{asset1}': amm.fees[asset1], f'F{asset2}': amm.fees[asset2], 'FL': amm.fees['L']}
                df = df.append(new_row, ignore_index=True)# append new row to df
        for amm, df in amms:
            sim_amm_dfs.append(df)
            sim_amms.append(amm)
    return sim_amm_dfs, sim_amms # return list of dfs for each simulation

In [98]:
# # NOTES FROM LAST MEETING:
# FOCUS MORE ON TESTING FEES THROUGH SIM

# # EXPERIMENTS TODO: # #
# [1] run for large simulations and evaluate over time - explore different time periods to test from (different market conditions and lengths of historical windows) and different frequencies (1h, 1d, 1w)
# [2] identify GBM paths that deplete pools (depletion of liquidity) and have both fall in value (impermanent loss) to show how fee accumulation compares ot general trend (law of large #s)
        # impermanent loss evaluation could allow for an expected value calculation for LP returns (expected value of fees vs. impermanent loss)
# [3] use stock data to see how compares
# [4] make sure to highlight how different fee AMMs (basically fees) are affected by different market conditions and therefore how fee accumulation is affected

# # UPDATES # #
# [1] *importing stock data to use instead of crypto (more in line with goal application and can properly use GBM to simulate)
# [2] considering train/test split for calibrating GBM and simulating trades source data (not overly urgent given not forecasting)
# [3] maybe also considering changing source data from vwap if stick with crypto data
        # multiple price streams for multiple external oracles

In [99]:

sim1(2, "btc-eth", '2023-02-01T00:00:00Z', '2024-03-01T00:00:00Z', "1d")

[{'timestamp': 1675209600000, 'price': '23182.447711684785'}, {'timestamp': 1675296000000, 'price': '23838.31985916336'}, {'timestamp': 1675382400000, 'price': '23460.165908583494'}, {'timestamp': 1675468800000, 'price': '23405.345681665356'}, {'timestamp': 1675555200000, 'price': '23083.502228648224'}, {'timestamp': 1675641600000, 'price': '22905.31987241254'}, {'timestamp': 1675728000000, 'price': '23028.9605071829'}, {'timestamp': 1675814400000, 'price': '23028.88024701231'}, {'timestamp': 1675900800000, 'price': '22353.184272519957'}, {'timestamp': 1675987200000, 'price': '21748.009849510887'}, {'timestamp': 1676073600000, 'price': '21708.802480513128'}, {'timestamp': 1676160000000, 'price': '21878.46823038625'}, {'timestamp': 1676246400000, 'price': '21646.959927678326'}, {'timestamp': 1676332800000, 'price': '21970.09973026002'}, {'timestamp': 1676419200000, 'price': '23256.27192103903'}, {'timestamp': 1676505600000, 'price': '24575.904743109313'}, {'timestamp': 1676592000000, 'p

TypeError: list indices must be integers or slices, not str